# Tuning Workflow
This notebook demonstrates how to extract compact feature windows and decisions for offline tuning and visualization.

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from smart_lighting_ai_dali.config import get_settings
from smart_lighting_ai_dali.models import FeatureRow, Decision

settings = get_settings()
engine = create_engine(settings.db_url)
with engine.connect() as conn:
    feature_df = pd.read_sql(FeatureRow.__table__.select(), conn)
    decision_df = pd.read_sql(Decision.__table__.select(), conn)

feature_df.tail()

In [ ]:
merged = feature_df.merge(decision_df, left_on='id', right_on='feature_row_id', how='left', suffixes=('_feature', '_decision'))
merged['energy_saving_estimate'].fillna((100 - merged['intensity'].fillna(100)) / 100, inplace=True)
merged[['window_end', 'ambient_lux_mean', 'occupancy_rate', 'intensity', 'cct', 'energy_saving_estimate']].tail()

In [ ]:
%matplotlib inline
ax = merged.plot(x='window_end', y='energy_saving_estimate', title='Energy Saving Estimate Over Time')
ax.set_ylabel('Energy Saving (fraction)')